# Quiz 2 #

**Nome:**  Edson Roteia e João Pedro Moreira Ferreira

### First estimation - Manual Calculating from single image###

- Configuration for naive estimation

![](./imgs/chess.jpg.png)

- Size in pixel of the book:
   - width: 870px
   - heigth: 696px
   

From the equations:
$f_x = \frac{d_x}{d_X} d_z, f_y = \frac{d_y}{d_Y} d_z $ (1)

We have:
$f_x = \frac{870}{294} 1200, f_y = \frac{696}{234}1200$

So:
$f_x = 3551.02, f_y = 3569.23$

### Second estimation - Using MATLAB App (Camera Calibrator)###

- Input images to estimate camera parameters

![](./imgs/input_matlab.png)


- Camera Calibrator App for MATLAB

![](./imgs/app.png)


- Camera parameters estimate by the MATLAB App

![](./imgs/matlab_5.png)

The Camera Calibrator MATLAB App Estimate the follow parameters:
   - Intrisic Parameters:
    
   - Extrinsic Parameters:

### Last Estimation - Using SVD 

#### 1. Measure the 3-D coordinates of each vector of the n squares on the calibration pattern in the world reference frame.

In [86]:
import cv2
import numpy as np

##read images
images_name = ['./imgs/IMG_20190404_153953657.jpg','./imgs/IMG_20190404_153954488.jpg','./imgs/IMG_20190404_154000947.jpg',
              './imgs/IMG_20190404_154002764.jpg','./imgs/IMG_20190404_154004200.jpg']
images = []
for img_name in images_name:
    img = cv2.imread(img_name)
    images.append(img)

##find chessboard
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 28, 0.001)
objp = np.zeros((5*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:5].T.reshape(-1,2)

objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.





# 8. Set up A and b of system (6.14), and use (6.15) to estimate T_{z} and f_{x}
#      - The output is formed by alpha, f_{x}, and the extrinsic parameters of the viewing camera.
for img in images:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,5), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
#         cv2.drawChessboardCorners(img, (7,5), corners2, ret)
#         cv2.imshow('img', cv2.resize(img,(800,800)))
#         cv2.waitKey()

#### 2. Find the coordinates in the image reference frame of each of the N vertices.

In [87]:
#####POINTS IN THE WORLD##########
worldpoints = []
#points in 1,2,3,4,5,6,7
# worldpoints = objpoints
#points i1n 0.028,0.056,0.084,0.112,0.140,0.168
for points in objpoints:
    worldpoints.append(points * 28)

#### 3. Having the N correspondences between image and world points, compute the SVD of A (6.8). The solution is column of V corresponding to the smallest singular value of A.
![](./imgs/trucco6.8.png)

In [88]:
# Create A matrix
A = []
for worldpoint_vector,imagepoint_vector in zip(worldpoints,imgpoints):
    for worldpoint, imagepoint in zip(worldpoint_vector, imagepoint_vector):
#         print(worldpoint, imagepoint)
        A_line = []
        A_line.append(imagepoint[0][0] * worldpoint[0])
        A_line.append(imagepoint[0][0] * worldpoint[1])
        A_line.append(imagepoint[0][0] * worldpoint[2])
        A_line.append(imagepoint[0][0])
        A_line.append(-imagepoint[0][1] * worldpoint[0])
        A_line.append(-imagepoint[0][1] * worldpoint[1])
        A_line.append(-imagepoint[0][1] * worldpoint[2])
        A_line.append(-imagepoint[0][1])
        A.append(A_line)
        
A = np.asarray(A)
U, D, V = np.linalg.svd(np.dot(np.transpose(A), A), full_matrices=True)
idx_min = np.argmin(D)
V_selected = V[idx_min]

#### 4. Determine $|\gamma|$ and $\alpha$ from (6.10) and (6.11)
![](./imgs/trucco6.10.png)
![](./imgs/trucco6.11.png)

In [89]:
gamma = (V_selected[0] ** 2 + V_selected[1] ** 2 + V_selected[2] ** 2 ) ** (1/2)
alfa = ((V_selected[4] ** 2 + V_selected[5] ** 2 + V_selected[6] ** 2 ) ** (1/2))/gamma
print(gamma, alfa)
print (idx_min,D,D[np.nonzero(D)],D[idx_min]<D[idx_min-1])
print ('-------')
print (V_selected)
print ("--------")
print (V[0][idx_min],V[1][idx_min],V[2][idx_min],V[3][idx_min])
print (V)

2.220446049250313e-16 4503599627370496.0
7 [1.8421947e+13 2.4578577e+12 2.7763533e+11 6.5202831e+10 1.8291214e+08
 4.7917330e+06 7.2991029e-06 0.0000000e+00] [1.8421947e+13 2.4578577e+12 2.7763533e+11 6.5202831e+10 1.8291214e+08
 4.7917330e+06 7.2991029e-06] True
-------
[ 0.0000000e+00 -3.2367213e-32 -2.2204460e-16  2.1155799e-30
 -1.2400870e-32 -4.6969601e-32  1.0000000e+00  4.8965264e-30]
--------
0.002952252 0.00293668 -0.0053269747 0.008837353
[[-8.0981839e-01 -4.3207550e-01  0.0000000e+00 -6.7540999e-03
   3.3716726e-01  2.0921016e-01  0.0000000e+00  2.9522520e-03]
 [ 4.9490729e-01 -7.3478556e-01  0.0000000e+00 -1.7526917e-03
  -3.9745711e-02  4.6213156e-01  0.0000000e+00  2.9366801e-03]
 [-2.6442564e-01 -3.1968462e-01 -1.1102230e-16 -5.6484286e-03
  -8.5931021e-01 -2.9900926e-01  0.0000000e+00 -5.3269747e-03]
 [-1.7110625e-01  4.1375825e-01 -1.6653345e-16 -1.0263478e-02
  -3.8246176e-01  8.0812520e-01  0.0000000e+00  8.8373534e-03]
 [-7.6032029e-03 -8.3580479e-04 -3.2265857e-16 

#### 5. Recover the first two rows of R and the two first components of T from (6.9)
![](./imgs/trucco6.9.png)

In [90]:
r = np.ones(shape=(3,3))
t = np.ones(shape=(3,1))
r[1][0] = V_selected[0]/gamma
r[1][1] = V_selected[1]/gamma
r[1][2] = V_selected[2]/gamma
t[1] = V_selected[3]/gamma
r[0][0] = (V_selected[4]/gamma)/alfa
r[0][1] = (V_selected[5]/gamma)/alfa
r[0][2] = (V_selected[6]/gamma)/alfa
t[0] = (V_selected[7]/gamma)/alfa
# print (r,t)

#### 6. Compute the third row of R as the vector product of the first two rows estimated in the previous step, and enforce the orthogonality constraint on the estimate of R through SVD decomposition.

In [91]:
#inner = 0 | dot = r[:][3]
r[2,:] = np.cross(r[0,:],r[1,:])

print (r)
print (r[0,:])
print("******************")
print (np.inner(r[2,:],r[0,:]),np.inner(r[2,:],r[1,:])) 
print("******************")

[[-1.24008703e-32 -4.69696010e-32  1.00000000e+00]
 [ 0.00000000e+00 -1.45768970e-16 -1.00000000e+00]
 [ 1.45768970e-16 -1.24008703e-32  1.80766210e-48]]
[-1.24008703e-32 -4.69696010e-32  1.00000000e+00]
******************
0.0 0.0
******************


#### 7. Pick a point for which $(x - \sigma_{x})$ is noticeably different from 0. If inequality (6.12) is satisfied, reverse the sign of the first two rows of R and the first two components of T
![](./imgs/trucco6.12.png)

In [92]:
x_aux = imgpoints[0][1][0][0] * (r[0][0] * worldpoints[0][1][0] + r[0][1] * worldpoints[0][1][1] +
                                    r[0][2] * worldpoints[0][1][2] + t[0])

print(r)
r[0,:] = -r[0,:]
r[1,:] = -r[1,:]
t[0:2] = -t[0:2]
print(r)

print(x_aux)

A_2 = []
for worldpoint_vector,imagepoint_vector in zip(worldpoints,imgpoints):
    for worldpoint, imagepoint in zip(worldpoint_vector, imagepoint_vector):
        A_line = []
        A_line.append(imagepoint[0][0])
        A_line.append((r[0][0] * worldpoint[0] + r[0][1] * worldpoint[1] + r[0][2] * worldpoint[2] + t[0])[0])
        A_2.append(A_line)

b = []
for worldpoint_vector,imagepoint_vector in zip(worldpoints,imgpoints):
    for worldpoint, imagepoint in zip(worldpoint_vector, imagepoint_vector):
        b.append(-imagepoint[0][0] * (r[2][0] * worldpoint[0] + r[2][1] * worldpoint[1] + r[2][2] * worldpoint[2]))

np.dot(np.dot(np.linalg.inv(np.dot(np.transpose(A_2), A_2)), np.transpose(A_2)), b)

[[-1.24008703e-32 -4.69696010e-32  1.00000000e+00]
 [ 0.00000000e+00 -1.45768970e-16 -1.00000000e+00]
 [ 1.45768970e-16 -1.24008703e-32  1.80766210e-48]]
[[ 1.24008703e-32  4.69696010e-32 -1.00000000e+00]
 [-0.00000000e+00  1.45768970e-16  1.00000000e+00]
 [ 1.45768970e-16 -1.24008703e-32  1.80766210e-48]]
[9.84574922e-27]


array([-1.59998674e-14, -3.83553920e+18])

### Fourth Estimation - OpenCV Calibration Function

In [93]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
print('mtx:',mtx,'\ndist:',
      dist,'\nrvecs',rvecs,'\ntvecs:',tvecs)

mtx: [[3.56361851e+03 0.00000000e+00 2.31043347e+03]
 [0.00000000e+00 3.56712174e+03 1.30610024e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] 
dist: [[ 3.62710629e-01 -4.05401065e+00  3.50476240e-03 -1.97307215e-03
   1.74055005e+01]] 
rvecs [array([[-0.56801087],
       [-0.0330287 ],
       [-0.01076532]]), array([[-0.3817585 ],
       [-0.03292341],
       [-0.00594361]]), array([[ 0.17415325],
       [-0.04947198],
       [-0.00576611]]), array([[ 0.18109886],
       [-0.00999371],
       [-0.02069352]]), array([[ 0.17898442],
       [-0.01176046],
       [-0.04216456]])] 
tvecs: [array([[-2.03098909],
       [-3.52434165],
       [25.29084783]]), array([[-2.14813411],
       [-3.78547298],
       [24.71926373]]), array([[-2.82271701],
       [-7.26621745],
       [23.80149558]]), array([[-2.67716662],
       [-4.12883513],
       [24.34515758]]), array([[-2.65868337],
       [ 1.14534746],
       [24.97406838]])]
